## 0. Import Library and load csv

CatBoost Quickstart

LB:0.92058

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv')

## 1. Data Processing

In [ ]:
label = LabelEncoder()
def feat_eng(df, test):
    if test:
        x = df.drop(columns = 'row_id')
        return x
    else:
        x = df.drop(columns = ['row_id', 'target'])
        y = df['target']
        y = label.fit_transform(y)
        y = pd.DataFrame(y)
        return x, y  
        

In [ ]:
train_x, train_y = feat_eng(train_df, test = False)
test_x = feat_eng(test_df, test= True)

## 2. Train model 

In [ ]:
from sklearn.ensemble  import RandomForestClassifier

kf = KFold(n_splits = 5, shuffle = True, random_state = 70)

# modeling and training
for fold, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
    print(f'--------fold:{fold+1}--------')
    fold+=1
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]                  
    model = RandomForestClassifier(n_estimators=50, n_jobs=-1)
    # Training the model
    model.fit(tr_x, tr_y)
    

    preds = model.predict(va_x)
    score = accuracy_score(va_y, preds)
    print(f'score: {score}')

In [ ]:
# # fold5
# kf = KFold(n_splits = 5, shuffle = True, random_state = 70)

# # modeling and training
# for fold, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
#     print(f'--------fold:{fold+1}--------')
#     fold+=1
#     tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
#     tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
#     params = { 'depth' : 9,
#             'random_strength' :0.03,       
#             'l2_leaf_reg': 11.107218435931566,               
#             'bagging_temperature' : 8.38565996766423,
#             'learning_rate' : 0.21949607909590932,  
#             'border_count' : 10,
#             'eval_metric': 'Accuracy',
#             'min_child_samples': 0.448931355740778
#             }

                  
#     model = CatBoostClassifier(**params,task_type="GPU")
#     # Training the model
#     model.fit(tr_x, tr_y, verbose=0, early_stopping_rounds=100)
    

#     preds = model.predict(va_x)
#     score = accuracy_score(va_y, preds)
#     print(f'score: {score}')

## 3. Submission

In [ ]:
sub_df = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
preds = model.predict(test_x)
preds = label.inverse_transform(preds)
sub_df['target'] = preds
sub_df.to_csv('submission.csv', index=False)